In [1]:
import requests
import pandas as pd
import io


# getEventStartLists

def get_csos_club_list():
    url = "https://oris.orientacnisporty.cz/API/"
    params = {
        'format': 'json',
        'method': 'getCSOSClubList'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful
    return response.json()

def download_and_load_csv(gender: str, date: str) -> pd.DataFrame:
    url = "https://oris.orientacnisporty.cz/ranking_export"
    params = {
        'date': date,
        'sport': '1',
        'gender': gender,
        'csv': '1'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful

    csv_data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(csv_data), sep=";")
    return df

def download_rankings(s_time1, s_time2) -> dict[str, list[pd.DataFrame]]:
  ranking_dfs = {}
  ranking_dfs["M"] = [download_and_load_csv("M", s_time1), download_and_load_csv("M", s_time2)]
  ranking_dfs["F"] = [download_and_load_csv("F", s_time1), download_and_load_csv("F", s_time2)]
  print("Ranking Data:")
  #_ = [print(x.describe()) for x in ranking_dfs["M"]]
  #_ = [print(x.describe()) for x in ranking_dfs["F"]]
  return ranking_dfs

def get_event_entries(event_id: int=8657):
    url = "https://oris.orientacnisporty.cz/API/"
    params = {
      'format': 'json',
      'method': 'getEventEntries',
      'eventid': f"{event_id}"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful
    return response.json()

In [2]:
PAST_RANKING = tuple[float, float, float]
RANKING = tuple[float, float, float, PAST_RANKING]


def get_ranking_by_regno(df: pd.DataFrame, minule_df: pd.DataFrame, reg_no: str) -> RANKING:
  filtered_df = df[df['Reg. c.'] == reg_no]
  minule_filtered_df = minule_df[minule_df['Reg. c.'] == reg_no]
  if filtered_df.empty:
    return (float('inf'), -1, -1, (float('inf'), float('inf'), float('inf')))
  if minule_filtered_df.empty:
    minule = filtered_df['Minule Ran'].iloc[0]
    return (
      filtered_df['Poradi'].iloc[0]
      , filtered_df['Body'].iloc[0]
      , filtered_df['Rank.c'].iloc[0]
      , (
          minule if minule > 0 else float('inf')
          , float('inf')
          , float('inf')
      )
    )
  return (
    filtered_df['Poradi'].iloc[0]
    , filtered_df['Body'].iloc[0]
    , filtered_df['Rank.c'].iloc[0]
    , (
        filtered_df['Minule Ran'].iloc[0]
        , minule_filtered_df['Poradi'].iloc[0]
        , minule_filtered_df['Minule Ran'].iloc[0]
    )
  )

In [51]:
from enum import Enum


class MultiRaceClass(Enum):
  H21E = "H21E"
  D21E = "D21E"
  H21A = "H21A"
  D21A = "D21A"
  H21B = "H21B"
  D21B = "D21B"
  H_OTHER = "H_OTHER"
  D_OTHER = "D_OTHER"
  OTHER = ""

class AcademicRaceClass(Enum):
  H = "H"
  D = "D"
  H_OTHER = "H_OTHER"
  D_OTHER = "D_OTHER"
  OTHER = ""

def class_2_gender(r: Enum) -> str:
  if r.value == "":
    return ""
  return "M" if r.value[0] == "H" else "F" if r.value[0] == "D" else ""

# get_gender_akademicky = get_gender[AcademicRaceClass]

def get_gender_multi(s: str) -> MultiRaceClass:
  for e in MultiRaceClass:
    if e.value == s:
      return e
  if "H" in s:
    return MultiRaceClass.H_OTHER
  elif "D" in s:
    return MultiRaceClass.D_OTHER
  return MultiRaceClass.OTHER

def get_gender_akademicky(s: str) -> AcademicRaceClass:
  if AcademicRaceClass.H.value == s:
    return AcademicRaceClass.H
  elif AcademicRaceClass.D.value == s:
    return AcademicRaceClass.D
  elif "H" in s:
    return AcademicRaceClass.H_OTHER
  elif "D" in s:
    return AcademicRaceClass.D_OTHER
  return AcademicRaceClass.OTHER

In [89]:
from dataclasses import dataclass
from typing import TypeVar, Type, Union


T = TypeVar('T', bound='Entry')
EntryUnion = Union['AcademicRaceClass', 'MultiRaceClass']
EntryTypes = Type[EntryUnion]

@dataclass
class Entry:
  classR: EntryTypes
  name: str
  regno: int
  licence: str
  ranking: RANKING

  @property
  def ranked(self) -> float:
    if self.ranking is None:
      return None
    return self.ranking[0]


@dataclass
class MultiEntry(Entry):
  classR: MultiRaceClass

  @staticmethod
  def get_gender(class_desc: str) -> 'MultiRaceClass':
    return get_gender_multi(class_desc)


@dataclass
class AcademicEntry(Entry):
  classR: AcademicRaceClass

  @staticmethod
  def get_gender(class_desc: str) -> 'AcademicRaceClass':
    return get_gender_akademicky(class_desc)


def divide_entries(entries: list[T], race_enum: EntryTypes) -> dict[EntryUnion, list[T]]:
  ret = {race_class: [] for race_class in race_enum}
  for entry in entries:
    ret[entry.classR].append(entry)
  for cls, ls in ret.items():
    ret[cls] = sorted(ls, key=lambda entry: entry.ranked)
  return ret

def mk_entries(entries: pd.DataFrame, ranking_dfs: dict[str, list[pd.DataFrame]], entry_type: Type[T]) -> list[T]:
  entrs: list[T] = []
  for x in entries["Data"]:
    obj = entries["Data"][x]
    cls = entry_type.get_gender(obj["ClassDesc"])
    gender = class_2_gender(cls)
    entrs.append(Entry(
        classR=cls
        , name=obj["Name"]
        , regno=obj["RegNo"]
        , licence=obj['Licence']
        , ranking=get_ranking_by_regno(ranking_dfs[gender][0], ranking_dfs[gender][1], obj["RegNo"])
        if cls != entry_type.__annotations__.get('classR').OTHER else None
    ))
  return entrs

In [90]:
def print_analyze(div_entrs: dict[Enum, list[Entry]], event_id: int) -> None:
  print(f"RANKING z ORISU, event #{event_id}")
  for r, ls in div_entrs.items():
    print(f"{r} ({len(ls)})")
    for x in ls:
      print(f"({x.licence}) {x.name.ljust(20)} \t{x.ranked}.\t{x.ranking[3]}.")

In [86]:
def analyze_event(event_id: int, ranking_dfs, entry_type: Entry, gender=None, age=None, level=None) -> None: # type hint
  entries = get_event_entries(event_id)

  enum_type = entry_type.__annotations__.get('classR')

  entrs = mk_entries(entries, ranking_dfs, entry_type)
  div_entrs = divide_entries(entrs, enum_type)

  div_entrs = filter_entries(enum_type, div_entrs, gender, age, level)

  print_analyze(div_entrs, event_id)

In [81]:
ranking_dfs = download_rankings('2024-05-31', '2024-03-31')

Ranking Data:


In [91]:
analyze_event(8657, ranking_dfs, AcademicEntry)

RANKING z ORISU, event #8657
AcademicRaceClass.H (43)
(E) Vandas Daniel        	5.	(7, 4, 4).
(E) Coufal Jáchym        	8.	(11, 12, 12).
(E) Janovský Tomáš       	10.	(14, 15, 15).
(E) Horvát Petr          	11.	(8, 8, 8).
(E) Metelka Ondřej       	13.	(10, 11, 11).
(E) Kostka Jiří          	16.	(52, 52, 94).
(E) Gajda Jan            	17.	(65, 23, 23).
(E) Adámek Filip         	18.	(19, 19, 19).
(R) Čech Vít             	20.	(23, 30, 36).
(E) Pompura Daniel       	23.	(21, 35, 35).
(E) Hájek Kryštof        	25.	(35, 16, 16).
(R) Škvor Ota            	34.	(22, 22, 21).
(R) Melišík Martin       	44.	(61, 72, 74).
(E) Donda Jiří           	52.	(55, 63, 73).
(R) Tokár Radim          	62.	(76, 86, 90).
(C) Janas Petr           	68.	(70, 74, 72).
(C) Panovec Kryštof      	75.	(102, 103, 103).
(C) Mareček Šimon        	79.	(63, 67, 66).
(C) Macek Ondřej         	84.	(77, 89, 88).
(C) Thoř Tomáš           	107.	(94, 96, 99).
(C) Pipek Ondřej         	129.	(113, 111, 110).
(C) Měšťan Ondřej     

In [93]:
from functools import partial

def filter_entries(enum, entries, gender, age, level): #type hint
  categories = get_x_y_categories(enum, gender, age, level)
  return { k: v for k, v in entries.items() if k in categories }

def category_cond(gender, age, level, enum_instance) -> bool:
  return (
      (True if gender is None else gender in enum_instance.value)
    and (True if age is None else f"{age}" in enum_instance.value)
    and (True if level is None else level in enum_instance.value)
  )

def get_x_y_categories(enum, gender, age, level): # type hint
  cond = partial(category_cond, gender, age, level)
  return [e for e in enum if cond(e)]

In [92]:
analyze_event(7609, ranking_dfs, MultiEntry, "H", 21, None)

RANKING z ORISU, event #7609
MultiRaceClass.H21E (76)
(E) Coufal Jáchym        	8.	(11, 12, 12).
(E) Kostka Jiří          	16.	(52, 52, 94).
(E) Gajda Jan            	17.	(65, 23, 23).
(E) Adámek Filip         	18.	(19, 19, 19).
(E) Richtr Lukáš         	19.	(20, 20, 20).
(E) Petržela Jan         	21.	(13, 13, 13).
(R) Poklop Martin        	22.	(25, 28, 27).
(E) Hájek Kryštof        	25.	(35, 16, 16).
(E) Kozák Osvald         	26.	(17, 17, 17).
(E) Rollier Baptiste     	27.	(18, 18, 18).
(R) Procházka David      	28.	(24, 25, 29).
(R) Hirš Otakar          	29.	(54, 113, 113).
(R) Kamenický Jeroným    	31.	(42, 41, 45).
(R) Škvor Ota            	34.	(22, 22, 21).
(R) Prášil Marek         	36.	(28, 27, 26).
(R) Malačka Milan        	40.	(36, 37, 39).
(R) Švadlena Michal      	41.	(40, 45, 50).
(R) Fürst Matouš         	42.	(32, 36, 37).
(R) Illner Vojtěch       	43.	(37, 47, 47).
(R) Melišík Martin       	44.	(61, 72, 74).
(R) Mokrý Stanislav      	45.	(45, 48, 46).
(R) Procházka Jan    